In [1]:
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.layers import Dense,Activation,Flatten,Input,Dropout,GlobalAveragePooling2D
from keras.models import Model
from keras.utils import np_utils
import random

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [22]:
base_path = "C:\\Users\sumeet\Desktop\machine-learning-online-2018\Datasets\Images\images"
dirs = os.listdir("C:\\Users\sumeet\Desktop\machine-learning-online-2018\Datasets\Images\images")

In [23]:
print(dirs)

['cats', 'dogs', 'horses', 'humans']


In [24]:
folder_path = ""

In [25]:
image_data = []

In [26]:
labels = []

In [30]:
label_dict = {'cats':0,'dogs':1,'horses':2,'humans':3}

In [31]:
for ix in dirs:
    path = os.path.join(base_path,ix)
    img_data = os.listdir(path)
    for im in img_data:
        img = image.load_img(os.path.join(path,im),target_size=(224,224))
        img_array = image.img_to_array(img)
        image_data.append(img_array)
        labels.append(label_dict[ix])

In [34]:
print(len(image_data), len(labels))

1414 1414


In [35]:
combined = list(zip(image_data, labels))
random.shuffle(combined)

image_data[:], labels[:] = zip(*combined)

In [36]:
X_train = np.array(image_data)
Y_train = np.array(labels)

print(X_train.shape,Y_train.shape)

(1414, 224, 224, 3) (1414,)


In [37]:
Y_train = np_utils.to_categorical(Y_train)
print(X_train.shape,Y_train.shape)

(1414, 224, 224, 3) (1414, 4)


In [54]:
model = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

c:\python36\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [56]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [44]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256,activation='relu')(av1)
d1 = Dropout(0.5)(fc1)
fc2 = Dense(4,activation='softmax')(d1)

model_new = Model(inputs=model.input, outputs= fc2)
model_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [57]:
adam = Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [58]:
for ix in range(len(model_new.layers)):
    print(ix, model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x0000019B413FDF28>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x0000019B413FD400>
2 <keras.layers.convolutional.Conv2D object at 0x0000019B413FDE80>
3 <keras.layers.normalization.BatchNormalization object at 0x0000019B413FA898>
4 <keras.layers.core.Activation object at 0x0000019B413FA748>
5 <keras.layers.pooling.MaxPooling2D object at 0x0000019B40806898>
6 <keras.layers.convolutional.Conv2D object at 0x0000019B406E6BA8>
7 <keras.layers.normalization.BatchNormalization object at 0x0000019B201CDE10>
8 <keras.layers.core.Activation object at 0x0000019B201CDEF0>
9 <keras.layers.convolutional.Conv2D object at 0x0000019B201FFB70>
10 <keras.layers.normalization.BatchNormalization object at 0x0000019B20231438>
11 <keras.layers.core.Activation object at 0x0000019B20253F60>
12 <keras.layers.convolutional.Conv2D object at 0x0000019B202EB7F0>
13 <keras.layers.convolutional.Conv2D object at 0x0000019B2034F6D8>
14 <keras.layers.normalizati

In [60]:
for ix in range(169):
    model_new.layers[ix].trainable = False
    
model_new.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model_new.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
hist = model_new.fit(X_train,Y_train,
                    shuffle = True,
                    batch_size = 16,
                    epochs = 5,
                    validation_split=0.20
                    )

Train on 1131 samples, validate on 283 samples
Epoch 1/5
1131/1131 [==============================] - 2627s 2s/step - loss: 0.9803 - acc: 0.6260 - val_loss: 0.3436 - val_acc: 0.8940
Epoch 2/5
1131/1131 [==============================] - 2976s 3s/step - loss: 0.4119 - acc: 0.8744 - val_loss: 0.2010 - val_acc: 0.9399
Epoch 3/5
1131/1131 [==============================] - 2583s 2s/step - loss: 0.2532 - acc: 0.9319 - val_loss: 0.1742 - val_acc: 0.9399
Epoch 4/5
1120/1131 [============================>.] - ETA: 1:13 - loss: 0.2019 - acc: 0.9500

In [ ]:
plt.figure(0)
plt.plot(hist.history['acc'],'b')
plt.plot(hist.history['val_acc'],'g')
plt.plot(hist.history['loss'],'black')
plt.plot(hist.history['val_loss'],'red')
plt.show()